## **Northeastern SMILE Lab - Recognizing Faces in the Wild**

## Background

The SMILE Lab at Northeastern focuses on the frontier research of applied machine learning, social media analytics, human-computer interaction, and high-level image and video understanding. Their research is driven by the explosion of diverse multimedia from the Internet, including both personal and publicly available photos and videos. They start by treating fundamental theory from learning algorithms as the soul of machine intelligence and arm it with visual perception.

Northeastern University has been developing an automatic kinship classifier since 2010. However, the widespread adoption of this technology has been hindered by two primary factors:

- Data Limitations: Existing image databases for kinship recognition are insufficient in size and diversity to accurately represent global familial relationships.

- Model Complexity: The complex interplay of genetic and environmental factors influencing facial features necessitates a more advanced model than traditional computer vision algorithms, which are typically designed for higher-level tasks like facial recognition or object classification.

## Objective

Using the provided competition data, the following model predicts the probability of two individuals being related (1) based solely on a pair of facial images. 

## **Preliminaries**: Install the following Python modules.

In [ ]:
# If they are not already installed, this cell installs the modules used in this solution.
%pip install collections
%pip install glob
%pip install kaggle
%pip install matplotlib
%pip install os
%pip install pandas
%pip install sklearn
%pip install torch
%pip install torch.nn
%pip install torch.optim
%pip install zipfile

## **Step 1**: Download the official Kaggle competition (FIW) data.

***Before executing the following cell***, please review section 7 of the rules: https://www.kaggle.com/competitions/recognizing-faces-in-the-wild/rules#7-competition-data. 

Note that all usages of FIW data should cite the following papers:

*Joseph P Robinson, Ming Shao, Hongfu Liu, Yue Wu, Timothy Gillis, and Yun Fu. "Visual Kinship Recognition of Families In the Wild" IEEE TPAMI Special Edition: Computational Face (2018).
Joseph P Robinson, Ming Shao, Handong Zhao, Yue Wu, Timothy Gillis, Yun Fu. "Recognizing Families In the Wild (RFIW): Data Challenge Workshop in conjunction with ACM MM 2017," ACM Multimedia Conference: Workshop on RFIW (2017).
Shuyang Wang, Joseph P Robinson, and Yun Fu. “Kinship Verification on Families in the Wild with Marginalized Denoising Metric Learning,” in IEEE Automatic Face and Gesture Recognition (2017).
Joseph P Robinson, Ming Shao, Yue Wu, and Yun Fu. “Families In the Wild (FIW): large-scale kinship image database and benchmarks." in ACM on Multimedia Conference (2016).*

In [ ]:
download_path = '_provided-data' # Intermediate directories are excluded recursively via. .gitignore (i.e., '_*/').
competition = 'recognizing-faces-in-the-wild' # Hosted here: https://www.kaggle.com/c/recognizing-faces-in-the-wild

In [ ]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

os.makedirs(download_path, exist_ok=True)
print('Downloading ' + competition + ' provided data into ' + download_path)
api = KaggleApi()
api.authenticate()
api.competition_download_files(competition, path = download_path)

In [ ]:
import os
import zipfile

def unzip(zip_path):
    dest_dir = '_' + os.path.basename(zip_path)[:-4]
    if (not os.path.exists(dest_dir)):
        os.makedirs(dest_dir, exist_ok=True)
        print('Decompressing ' + zip_path + ' into ' + dest_dir)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(dest_dir)
    print(dest_dir + '/')
    print(os.listdir(dest_dir))
    return dest_dir

print('Decompressing ' + competition + ' provided data:')
dataset_path = unzip(download_path + '/' + competition + '.zip')
training_image_path = unzip(dataset_path + '/train-faces.zip')
testing_image_path = unzip(dataset_path + '/test-faces.zip')

## **Step ?**: Preprocess the image portion of the provided dataset.

In [ ]:
from collections import defaultdict
import glob

# Create a dictionary to lookup image files for each member
family_dict = defaultdict(list)
for family in glob.glob(training_image_path + '*'):
    for member in glob.glob(family + '/*'):
        for image_path in glob.glob(member + '/*'):
            member = member.split('\\')[-1]
            image_path = image_path.split('\\')[-1]
            family_dict[member].append(image_path)

print('Here are the first few of the ' + str(len(family_dict.items())) + ' total families:')
for key, value in list(family_dict.items())[:5]:
    print(str(key) + ': ' + str(value) + ',')

## **Step 4**: Preprocess (clean) the relational portion of the provided dataset.

The _train_relationships.csv_ encodes (ground truth) kinship relations between pairs of people. 

In [ ]:
import pandas
relations_csv_path = dataset_path + '/train_relationships.csv'
print('Loading ' + relations_csv_path)
relations_df = pandas.read_csv(relations_csv_path, delimiter=',', header='infer')
relations_df.head()

## **Step ?**: Remove invalid encodings
To improve the training efficiency, the relations dataframe is groomed to be fully valid (each referenced family member has at least one images in train-faces\...).

In [ ]:
import pandas
relations_csv_path = dataset_path + '/train_relationships.csv'
print('Loading ' + relations_csv_path)
relations_df = pandas.read_csv(relations_csv_path, delimiter=',', header='infer')

# Remove entries which do not exist in the training set
print('The original ground truth relations data contains ' + str(len(relations_df)) + ' entries.')
print(relations_df)

print('Checking for missing relation image data.')
fam_keys = family_dict.keys()
missing_relations_list = []
for index, row in relations_df.iterrows():
    split1 = row.p1.split('/')
    split2 = row.p2.split('/')
    p1fam = split1[0]
    p2fam = split2[0]
    if (p1fam not in fam_keys or p2fam not in fam_keys):
        missing_relations_list.append(index)
        continue
    p1member = split1[1]
    p2member = split2[1]
    if (p1member not in family_dict[p1fam] or p2member not in family_dict[p2fam]):
        missing_relations_list.append(index)
        continue
    images1 = os.listdir(training_image_path + '/' + p1fam + '/' + p1member)
    images2 = os.listdir(training_image_path + '/' + p2fam + '/' + p2member)
    if (len(images1) == 0 or len(images2) == 0):
        missing_relations_list.append(index)
        continue
if (missing_relations_list.count):
    relations_df = relations_df.drop(missing_relations_list)
    print(str(len(missing_relations_list)) + ' entries were removed due to missing data.')
    print(relations_df)
else:
    print('All relations were found to have valid image data.')


## **Step ?**: Create a lookup function to return if members are related.

In [ ]:
# This utility function to check if two members are related, regardless of the provided order.
def isRelated(df, p1, p2):
    return (df[(df['p1'] == p1) & (df['p2'] == p2)].shape[0] > 0) \
        or (df[(df['p1'] == p2) & (df['p2'] == p1)].shape[0] > 0)

# Show a couple of true negatives and true positives
print(isRelated(relations_df, 'bad', 'data'))              # False
print(isRelated(relations_df, 'F0002/MID2', 'F0005/MID2')) # False
print(isRelated(relations_df, 'F0998/MID6', 'F0998/MID3')) # True
print(isRelated(relations_df, 'F0998/MID3', 'F0998/MID6')) # True


## **Step ?**: Build a list of ground truth members and images

In [ ]:
# Create a list of all of the members in the relations data
members = []
for member in relations_df.values:
    if (member[0] not in members):
        members.append(member[0])
    if (member[0] not in members):
        members.append(member[0])
members.sort()
print(str(len(members)) + ' total members in the training set:')
print(members)

# Create a list of the corresponding images
image_set = []
for member in members:
    set = os.listdir(training_image_path + '/' + member)
    if (len(set) > 0):
        for image in set:
            image_set.append(member + '/' + image)
print(str(len(image_set)) + ' total images in the training set:')
print(image_set)

In [ ]:
import random

# Use a a portion of the images for training (training_images)...
training_images = random.sample(image_set, int(len(image_set)               -100))
print(str(len(training_images)) + ' images in training_images')
training_images.sort()
print('Showing the first three (sorted):')
print(training_images[:3])
print('Showing the first three (after shuffling):')
random.shuffle(training_images)
print(training_images[:3])

# ...and the remaining for testing (test_images).
test_images = [x for x in image_set if x not in training_images]
print(str(len(test_images)) + ' images in test_images')
print('Showing the first three (sorted):')
print(test_images[:3])
random.shuffle(test_images)
print('Showing the first three (after shuffling):')
print(test_images[:3])


## **Step ?**: Test that at least one image exists for a random selection of the relations.

In [ ]:
import cv2
import matplotlib.pyplot as plt
from random import choice

SAMPLE_COUNT = 3
samples = relations_df.sample(SAMPLE_COUNT)
print(samples)

f, ax = plt.subplots(SAMPLE_COUNT, 2)
f.patch.set_alpha(0) # Transparent background
for a in ax.flat:
    a.axis('off')
i = 0
for member in samples.values:
    img1 = training_image_path + '/' + member[0] + '/' \
        + choice(os.listdir(training_image_path + '/' + member[0]))
    img2 = training_image_path + '/' + member[1] + '/'  \
        + choice(os.listdir(training_image_path + '/' + member[1]))
    rgb1 = cv2.cvtColor(cv2.imread(img1), cv2.COLOR_BGR2RGB)
    rgb2 = cv2.cvtColor(cv2.imread(img2), cv2.COLOR_BGR2RGB)
    ax[i][0].imshow(rgb1, cmap = plt.cm.Spectral)
    ax[i][1].imshow(rgb2, cmap = plt.cm.Spectral)  
    i = i + 1

## **Step ?**: Train the model

The training process:

1. Pass the first image of the image pair through the network.
2. Pass the 2nd image of the image pair through the network.
3. Calculate the loss using the results of the prior steps.
4. Back propagate the loss to calculate the gradients.
5. Update the weights using an optimizer (Adam).

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()

        # Convolutional layers
        self.cnn1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )

        # Fully connected layers
        self.fc1 = nn.Sequential(
            nn.Linear(64 * 12 * 12, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128)
        )

    def forward_once(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

# Define the contrastive loss function
def contrastive_loss(output1, output2, label):
    euclidean_distance = torch.sqrt(torch.sum((output1-output2)**2, dim=1, keepdim=True))
    loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                  (label) * torch.pow(torch.clamp(2-euclidean_distance, min=0.0), 2))
    return loss_contrastive

# Create the Siamese network
model = SiameseNetwork()

# Define the optimizer and loss function
criterion = contrastive_loss
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Training loop
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0

    for i, (data1, data2, label) in enumerate(dataloader):
        data1, data2, label = data1.to(device), data2.to(device), label.to(device)

        optimizer.zero_grad()

        output1, output2 = model(data1, data2)
        loss = criterion(output1, output2, label)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * data1.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    print(f'Epoch Loss: {epoch_loss:.4f}')



## **Step ?**: Test the model

In [ ]:
import itertools

MODEL = False
if (MODEL):
    # Create a list of all of the image pairs in the test-faces data
    test_pairs = list(itertools.combinations(os.listdir(testing_image_path), 2))
else:
    # For now, use a portion of the ground truth relations data for testing
    test_pairs = list(itertools.combinations(test_images, 2))
    
    # Remove invalid pairs (same member)
    invalid_pairs = []
    i = 0
    for pair in test_pairs:
        person1fam_index = pair[0].find('/')
        person2fam_index = pair[1].find('/')
        person1 = pair[0][:pair[0].find('/', person1fam_index+1)]
        person2 = pair[1][:pair[1].find('/', person2fam_index+1)]
        if (person1 == person2):
            invalid_pairs.append(i)
        i = i + 1
    test_pairs = [x for i, x in enumerate(test_pairs) if i not in invalid_pairs]
    print(str(len(invalid_pairs)) + ' pairs of the same member were removed.')
    
print('There are ' + str(len(test_pairs)) + ' total pairs in the test set.')


In [ ]:
# Inference function
def predict_related(img1, img2):
    likelihood = (path1[:path1.find('/')] == path2[:path2.find('/')])
    return int(likelihood)*0.999
    # TODO Once the model is trained, use it instead of the above placeholder
    # TODO Preprocess the images
    output1, output2 = model(img1, img2)
    distance = torch.sqrt(torch.sum( (output1 - output2)**2, dim=1, keepdim = True ))
    return 1-distance
        
results = []
print(len(test_pairs))
for pair in test_pairs:
    person1fam_index = pair[0].find('/')
    person2fam_index = pair[1].find('/')
    person1 = pair[0][:pair[0].find('/', person1fam_index+1)]
    person2 = pair[1][:pair[1].find('/', person2fam_index+1)]
    results.append({'actual':int(isRelated(relations_df, person1, person2)),
                    'prediction':float(predict_related(pair[0], pair[1]))})


results_df = pandas.DataFrame(results)
results_df.describe()

## **Step ?**: Evaluation
Submissions are [evaluated](https://www.kaggle.com/competitions/recognizing-faces-in-the-wild/overview/evaluation) on area under the ROC curve between the predicted probability and the observed target. Not all pairs will be scored.

**Submission File**

For each img_pair in the test set, you must predict a probability for the is_related variable. The column img_pair describes the pair of images, i.e., abcdef-ghijkl means the pair of images abcdef.jpg and ghijkl.jpg.
The file should contain a header and have the following format:

```
img_pair,is_related
X3Nk6Hfe5x-qcZrTXsfde,0.0
X3Nk6Hfe5x-LD0pWDM8w_,0.0
X3Nk6Hfe5x-PHwuDtHyGp,0.0
X3Nk6Hfe5x-LO6lN_U4ot,0.0
...
```

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# https://en.wikipedia.org/wiki/Receiver_operating_characteristic
def generate_roc_curve(data):
  true_labels = [d['actual'] for d in data]
  predicted_probs = [d['prediction'] for d in data]
  fpr, tpr, _ = roc_curve(true_labels, predicted_probs)
  roc_auc = auc(fpr, tpr)
  plt.figure()
  plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.2f)' % roc_auc)
  plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver Operating Characteristic (ROC) Curve')
  plt.legend(loc="lower right")
  return plt
    
generate_roc_curve(results).show()

In [ ]:
import csv

submission = [['img_pair', 'is_related']]
for pair,result in zip(test_pairs, results):
    submission.append([pair[0] + '-' + pair[1], str(result['prediction'])])
filename = "submission.csv"
with open(filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerows(submission)
print(f"Saved '{filename}'")

import pandas
print('Loading ' + filename)
submission_df = pandas.read_csv(filename, delimiter=',', header='infer')
submission_df.describe()